# Pumas NCA Tutorial - Multiple dose IV administration
### Beatriz Guglieri lopez
### April 12, 2019

In [ ]:
using Pumas, PumasTutorials, CSV

# Introduction

In this tutorial, we will cover the fundamentals of performing an NCA analysis
with Pumas of an example dataset in which multiple intravenous (IV) bolus doses were administered.

# The dataset

* Four IV bolus doses of 2000 mg were administered every 24 hours to 24 different subjects.
* Samples were collected every 30 minutes.

Let's start reading the dataset.
By using the `missingstring` option we are specifying how the missing values are labeled in our dataset.

In [ ]:
data = PumasTutorials.tutorial_data("data/nca","multiple_dose_IVbolus_7BLQ_test")
data = CSV.read(data,missingstring="NA")
first(data,10)

This will be an abbreviated tutorial as the main difference is in the specification of the `read_nca` function.
For a complete listing of all NCA options, please check the first tutorial on single oral dose administration

# Defining the units

In [ ]:
timeu = u"hr"
concu = u"mg/L"
amtu  = u"mg"

# Defining the population object

The standard requirements of `read_nca` as specified in other tutorials exist. In this mulitple dose example, since subjects
visit in more than once occasion, we need to use an `occasion=` variable to speify the multiple dose nature.

In [ ]:
pop = read_nca(data, id=:ID, time=:time, conc=:DV, amt=:DOSE, ii=24timeu,
    route=:Formulation, occasion=:OCC,timeu=timeu, concu=concu, amtu=amtu,llq=0.4concu)

Key features of the syntax above:

* `route=` is mapped to the `Formulation` column that should specify `ev`
* LLOQ was set to 0.4 by `llq=0.4concu`
* `occasion=:OCC` provides a way to specify multiple dosing

To check how occasion works, lets calculate the AUC

In [ ]:
NCA.auc(pop,auctype=:last,method=:linear)

All other NCA function work on this grouped variable. Let's directly print the NCA report.

In [ ]:
report = NCAReport(pop)
report = NCA.to_dataframe(report)

Below is a listing of all the NCA parameters in the report

In [ ]:
names(report)

Finally, we can save this data frame as a csv file if desired.

In [ ]:
CSV.write("./tutorials/nca/report_MD_IVbolus_7BLQ.csv", report)